In [1]:
import json 
from pprint import pprint
from llm_handler import LLMHandler
from config import FILE_PATHS
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
domain = "telecom"
category = "adaptive_tool_use"

persona_path = FILE_PATHS["personas"].format(domain=domain)
scenario_path = FILE_PATHS["scenarios"].format(domain=domain, category=category)
tool_path = FILE_PATHS["tools"].format(domain=domain)

In [5]:
model_name = "claude-opus-4-20250514"
model_name = "gpt-4.1-mini-2025-04-14"

with open(scenario_path, "r") as f:
    scenarios = json.load(f)
    
with open(tool_path, "r") as f:
    tools = json.load(f)

#delete response_schema key from tool and keep rest of the keys
for tool in tools:
    tool.pop("response_schema", None)

llm = LLMHandler().get_llm(model_name=model_name, tools=tools)

In [6]:
# call the llm with the tools
system_prompt = """
You are a helpful assistant that can use the tools given to help the user. You can use multiple tools to answer the user's query. Generate multiple tool calls a time if needed.
"""

query = scenarios[0]["first_message"] 
query = "What is the balance of my account with the ids 123 and 234"
# query = "i want to check all the credit transactions for my account with the id 1234567890"
# query = "I am earning 1000000 per year. I need a home loan of 100000 for 1 year"

pprint(f"Query: {query}")
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query},
]

response = llm.invoke(messages)

print("\nModel Name:", model_name)
print("\nResponse Content:")
pprint(response.content)
print("\nTool Calls:")
pprint(response.tool_calls)
print("\nUsage Metadata:")
pprint(response.usage_metadata)
input_tokens, output_tokens = LLMHandler.get_token_usage_info(response)
print(f"\nToken Usage: input_tokens={input_tokens}, output_tokens={output_tokens}")

'Query: What is the balance of my account with the ids 123 and 234'

Model Name: gpt-4.1-mini-2025-04-14

Response Content:
''

Tool Calls:
[{'args': {'account_number': '123'},
  'id': 'call_2EuinOWISa73Ciu4Mjx5D4sD',
  'name': 'check_account_balance',
  'type': 'tool_call'},
 {'args': {'account_number': '234'},
  'id': 'call_sScTh5aGHCjGgXQ6cSJ8JB56',
  'name': 'check_account_balance',
  'type': 'tool_call'}]

Usage Metadata:
{'input_token_details': {'audio': 0, 'cache_read': 0},
 'input_tokens': 3832,
 'output_token_details': {'audio': 0, 'reasoning': 0},
 'output_tokens': 48,
 'total_tokens': 3880}

Token Usage: input_tokens=3832, output_tokens=48
